<a href="https://colab.research.google.com/github/ferids55/Coursera_Capstone/blob/main/Exploring_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Scraping Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [109]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## 1. Scraping Dataset


I'm scraping Toronto neighborhood data using this link https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and for get the location latitude and longitude each neighborhood using this link https://cocl.us/Geospatial_data. Last, joining two results into one dataframe.


In [110]:
# web scraping
link_data = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(link_data).text
soup = BeautifulSoup(html_data, 'html.parser')

table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# convert into dataframe
df_neighborhood = pd.DataFrame(table_contents)
df_neighborhood['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                    'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                    'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                    'MississaugaCanada Post Gateway Processing Centre':'Mississauga'}, inplace=True)
df_neighborhood.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Next, get geo location neighborhood data.


In [111]:
# Merge neighborhood data and geo location data
df_geoloc = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
df_geoloc.reset_index(inplace=True)
df_geoloc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Next, merge neighborhood data and geo location data by `Postal Code`.


In [112]:
toronto_data = pd.merge(df_neighborhood, df_geoloc, on='Postal Code')
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


And make sure that the dataset has all 15 boroughs and 103 neighborhoods.


In [113]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]))

The dataframe has 15 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>tor_explorer</em>, as shown below.


In [114]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto.


In [115]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


However, for illustration purposes, let's simplify the above map and segment and cluster only the toronto_data in **Scarborough**. So let's slice the original dataframe and create a new dataframe of the Scarborough data.


In [116]:
scarborough_data = toronto_data[toronto_data['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let's get the geographical coordinates of Scarborough.


In [117]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


As we did with all of Toronto, let's visualization Scarborough the neighborhoods in it.


In [118]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [119]:
CLIENT_ID = 'KCY5DQ3U3IWJXJDX13AOBZRAQ4MRBN3Z2FFHSANTJPSEMGFH' # your Foursquare ID
CLIENT_SECRET = '3JAORI5ILUFL14NIAUTVVOMFXEOLUTDYHHGWUBPV4QZVEB54' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KCY5DQ3U3IWJXJDX13AOBZRAQ4MRBN3Z2FFHSANTJPSEMGFH
CLIENT_SECRET:3JAORI5ILUFL14NIAUTVVOMFXEOLUTDYHHGWUBPV4QZVEB54


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [120]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.


In [121]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Malvern, Rouge within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [122]:
# type your answer here
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KCY5DQ3U3IWJXJDX13AOBZRAQ4MRBN3Z2FFHSANTJPSEMGFH&client_secret=3JAORI5ILUFL14NIAUTVVOMFXEOLUTDYHHGWUBPV4QZVEB54&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Send the GET request and examine the results


In [123]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60614fcec04a4d1ba832793e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [124]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [125]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


## 2. Explore Neighborhoods in Scarborough


#### Let's create a function to repeat the same process to all the neighborhoods in Scarborough


In [126]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _scarborough_venues_.


In [127]:
# type your answer here
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude'])
scarborough_venues.head()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


#### Let's check the size of the resulting dataframe


In [128]:
print(scarborough_venues.shape)

(89, 7)


Let's check how many venues were returned for each neighborhood


In [129]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues


In [130]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [131]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Gym,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [132]:
scarborough_onehot.shape

(89, 56)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [133]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Gym,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.000,0.250000,0.000000,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.25,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.000000,0.25,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.00,0.125,0.00,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.00,0.125,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.125000,0.000000,0.125000,0.00,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.00,0.000,0.00,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.083333,0.000000,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.083333,0.083333,0.000000,0.083333,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.083333,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000,0.00,0.000000,0.083333,0.000000,0.000000,0.166667,0.166667,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.00,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.000000,0.000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.142857,0.000000,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.000,0.000000,0.285714,0.000000,0.000000,0.000000,0.00,0.142857,0.00,0.000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.00,0.000,0.00,0.222222,0.000000,0.0,0.000000,0.111111,0.111111,0.00,0.000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.111111,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.00,0.000,0.111111,0.000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.00,0.000,0.000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000
7,"Guildwood, Mornin

#### Let's confirm the new size


In [134]:
scarborough_grouped.shape

(16, 56)

#### Let's print each neighborhood along with the top 5 most common venues


In [135]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3         Chinese Restaurant  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4           Noodle House  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.12
1  Caribbean Restaurant  0.12
2    Athletics & Sports  0.12
3       Thai Restaurant  0.12
4                Bakery  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                venue  freq
0         Pizza Place  0.17
1            Pharmacy  0.17
2  Italian Restaurant  0.08
3     Thai Restaurant  0.08
4        Noodle House  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1         Skat

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Chinese Restaurant,Vietnamese Restaurant,College Stadium,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Hakka Restaurant,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Bar,General Entertainment
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Pharmacy,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Chinese Restaurant,Bank,Gas Station,Italian Restaurant,Noodle House
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Athletics & Sports,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 4 clusters.


In [138]:
# set number of clusters
kclusters = 4
scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [139]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = pd.merge(scarborough_merged, neighborhoods_venues_sorted, on='Neighborhood')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Bank,Restaurant,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Mexican Restaurant,Electronics Store,Discount Store,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Korean BBQ Restaurant,Gym,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Bar,General Entertainment


Finally, let's visualize the resulting clusters


In [140]:
scarborough_merged.dropna(inplace=True)

In [141]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [142]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


#### Cluster 2


In [143]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Bank,Restaurant,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Mexican Restaurant,Electronics Store,Discount Store,Fast Food Restaurant
4,Scarborough,1,Hakka Restaurant,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Bakery,Athletics & Sports,Bar,General Entertainment
7,Scarborough,1,Bakery,Metro Station,Park,Soccer Field,Ice Cream Shop,Intersection,Bus Line,Bus Station,Construction & Landscaping,Convenience Store
8,Scarborough,1,American Restaurant,Skating Rink,Motel,Athletics & Sports,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
9,Scarborough,1,General Entertainment,Skating Rink,Café,College Stadium,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Gaming Cafe,Light Rail Station,Chinese Restaurant,Bank,Convenience Store,Asian Restaurant,Fried Chicken Joint
11,Scarborough,1,Auto Garage,Sandwich Place,Bakery,Middle Eastern Restaurant,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,Scarborough,1,Latin American Restaurant,Breakfast Spot,Lounge,Chinese Restaurant,Vietnamese Restaurant,College Stadium,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,1,Pizza Place,Pharmacy,Fried Chicken Joint,Thai Restaurant,Fast Food Restaurant,Chinese Restaurant,Bank,Gas Station,Italian Restaurant,Noodle House
14,Scarborough,1,Asian Restaurant,Intersection,Playground,Park,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 3


In [144]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,2,Coffee Shop,Korean BBQ Restaurant,Gym,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,Scarborough,2,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,Scarborough,2,Coffee Shop,Discount Store,Department Store,Convenience Store,Vietnamese Restaurant,Gas Station,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


#### Cluster 4


In [145]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop
